In [2]:
import pandas as pd
import glob
import json
import re
import numpy as np
from IPython.display import display
set_fn = set()


In [3]:

def extract_config(line):
    line = line.rstrip(')\n')
    config = re.findall("(\w+)='?([^,']*)'?", line)                                                                                                                                                             
    return {k: v for k, v in config}             

def list_dict_reverse(L):
    """ Given a list of dict with identical keys, turn that into a dict of numpy arrays.
    """
    keys = L[0].keys()
    res = {}
    for k in keys:
        res[k] = np.asarray([l[k] for l in L])
    return res

def parse_log(fn):
    with open(fn, 'r') as f:
        cfg = extract_config(f.readline())
        train_logs, test_logs = [], []
        for line in f.readlines():
            try:
                parsed = json.loads(line)
            except json.JSONDecodeError:
                continue
            if parsed['mode'] == 'train':
                train_logs.append(parsed)
            elif parsed['mode'] == 'test':
                test_logs.append(parsed)
        train_logs = list_dict_reverse(train_logs)
        test_logs = list_dict_reverse(test_logs)
        return cfg, train_logs, test_logs
        
data = []
data_var = []
for fn in glob.glob('../res_proto_*_adam/*'):
    print("Read", fn)
    if fn in set_fn:
        print("Already parsed", fn)
        continue
    set_fn.add(fn)
    try:
        cfg, tr, te = parse_log(fn)
    except:
        continue
    cfg.update({
        'final_length': te['length'][-1],
        'final_loss': te['loss_objs'][-1],
    })
    for k in te.keys():
        if k.startswith('length'):
            cfg.update({
                'init_' + k: te[k][0],
                'final_' + k: te[k][-1],
            })
    data_var.append(cfg)
df = pd.DataFrame(data_var)

Read ../res_proto_1_adam/142fcf07581a9459f21abf195f933ea7
Read ../res_proto_1_adam/9618e8245feb89bfd0e73dff270c3814
Read ../res_proto_1_adam/9cbd86b3c17143da6124d30f5f10db05_I
Read ../res_proto_1_adam/faf935b6feddaa27bf036e044ecefbbb_I
Read ../res_proto_1_adam/6769d503c18ebe0368f4a67034742502_I
Read ../res_proto_1_adam/e8544559fefced4d696acf8ab4b46c46
Read ../res_proto_1_adam/e6c4832633b6c2faec1cfdbc00681325
Read ../res_proto_1_adam/de26b589d6fe76cd9e3f0bb87a25a16b_I
Read ../res_proto_1_adam/959df9b0e90b362774dc63813eedbb1a_I
Read ../res_proto_1_adam/6769d503c18ebe0368f4a67034742502
Read ../res_proto_1_adam/4f8a3ae5727588bc063e46ae030f59ad
Read ../res_proto_1_adam/a73525cf78fea5cbee524b69d947469f
Read ../res_proto_1_adam/288bc39036389bdd2046da356de2a37f
Read ../res_proto_1_adam/590074bce1b184e6d94648127f33c755_I
Read ../res_proto_1_adam/092ca670df728ebfb5e54f2027555657
Read ../res_proto_1_adam/6d997e355f17f9d1ae02df734c5c791a
Read ../res_proto_1_adam/38e6f0d68cec738b9591168afea5d334
Re

In [4]:
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore', downcast='float'))


# Factors

## Temperature
High temperature seems higher accuracy, and lower final entropy for -1...

In [5]:
# fields = ['temperature', 'optimizer', 'lr', 'mode', 'random_seed', ]
fields = ['final_loss', 'final_length', 'lr', 'random_seed', 'optimizer', 'temperature']

criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
print(len(df.loc[criterion_1]))
a = df.loc[criterion_1][fields].groupby(['temperature'])
print(len(a))
print(a)
a.agg(['mean', 'std', 'count'])

58
4


final_loss                  final_length                  \
                  mean        std count         mean       std count   
temperature                                                            
0.90         13.199787   7.348947    17     5.042034  1.640664    17   
1.00         21.568178  10.715127    15     3.873485  2.421168    15   
1.25         13.317557   8.338977    12     5.061776  1.411581    12   
1.50         14.023396   8.859455    14     4.553142  2.332278    14   

                   lr                 random_seed                  
                 mean       std count        mean       std count  
temperature                                                        
0.90         0.000288  0.000285    17    1.882353  1.111438    17  
1.00         0.000413  0.000311    15    0.866667  1.125463    15  
1.25         0.000517  0.000361    12    1.416667  1.378954    12  
1.50         0.000471  0.000354    14    1.500000  1.091928    14

## Optimisation

- Use 3e-4 or even less, maybe?
- It's hard to conclude about beta2

In [6]:

fields = ['final_loss', 'final_length', 'lr', 'adam_beta2', 'random_seed', 'optimizer', 'temperature', 'batch_size']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['optimizer', 'lr', 'adam_beta2'])
display(a.agg(['mean', 'std', 'count']))

a = df.loc[criterion_1][fields].groupby(['optimizer', 'lr', 'batch_size'])
display(a.agg(['mean', 'std', 'count']))

a = df.loc[criterion_1][fields].groupby(['optimizer', 'batch_size', 'adam_beta2'])
display(a.agg(['mean', 'std', 'count']))

final_loss                  final_length  \
                                  mean        std count         mean   
optimizer lr     adam_beta2                                            
adam      0.0001 0.900        9.201864   1.225841     7     5.368124   
                 0.990       10.170705   1.350236     6     5.582108   
          0.0003 0.900       12.568557   8.091547    10     4.386564   
                 0.990       15.090096   7.135118     8     4.750000   
                 0.999       14.318420   6.960891    14     5.132066   
          0.0010 0.900       16.826181  12.946056     2     3.500000   
                 0.990       31.213291   7.765552     6     3.500000   
                 0.999       22.164740  12.865035     5     3.103610   

                                            random_seed                  \
                                  std count        mean       std count   
optimizer lr     adam_beta2                                               
adam      0.0001 0.900       0.692190     7    1.714286  1.603567     7   
                 0.990       0.656555     6    2.333333  0.516398     6   
          0.0003 0.900       2.001992    10    0.100000  0.316228    10   
                 0.990       2.314550     8    1.500000  1.309307     8   
                 0.999       1.742266    14    1.500000  1.160239    14   
          0.0010 0.900       3.535534     2    2.500000  0.707107     2   
                 0.990       2.738613     6    1.666667  1.211060     6   
                 0.999       2.652476     5    1.600000  0.547723     5   

                            temperature                  batch_size  \
                                   mean       std count        mean   
optimizer lr     adam_beta2                                           
adam      0.0001 0.900         0.978571  0.128638     7   86.857140   
                 0.990         1.100000  0.309839     6   96.000000   
          0.0003 0.900         1.240000  0.219596    10   89.599998   
                 0.990         1.137500  0.302076     8   92.000000   
                 0.999         1.110714  0.203033    14   73.142860   
          0.0010 0.900         1.375000  0.176777     2   80.000000   
                 0.990         1.150000  0.223607     6   80.000000   
                 0.999         1.230000  0.277489     5  128.000000   

                                              
                                   std count  
optimizer lr     adam_beta2                   
adam      0.0001 0.900       51.314158     7  
                 0.990       49.574187     6  
          0.0003 0.900       49.574187    10  
                 0.990       49.684720     8  
                 0.999       49.301049    14  
          0.0010 0.900       67.882251     2  
                 0.990       52.581366     6  
                 0.999        0.000000     5

final_loss                  final_length  \
                                  mean        std count         mean   
optimizer lr     batch_size                                            
adam      0.0001 32.0         9.691207   1.992012     5     5.256551   
                 128.0        9.622655   0.868289     8     5.598346   
          0.0003 32.0        14.678879   7.095024    15     5.223351   
                 128.0       13.334179   7.448763    17     4.433195   
          0.0010 32.0        33.189716   5.690288     4     3.500000   
                 128.0       22.110771  11.704962     9     3.279783   

                                            adam_beta2                  \
                                  std count       mean       std count   
optimizer lr     batch_size                                              
adam      0.0001 32.0        0.492513     5   0.936000  0.049295     5   
                 128.0       0.741455     8   0.945000  0.048107     8   
          0.0003 32.0        1.758921    15   0.970800  0.044335    15   
                 128.0       2.061814    17   0.961412  0.046899    17   
          0.0010 32.0        2.886751     4   0.967500  0.045000     4   
                 128.0       2.585819     9   0.985000  0.032171     9   

                            random_seed                 temperature            \
                                   mean       std count        mean       std   
optimizer lr     batch_size                                                     
adam      0.0001 32.0          1.400000  1.341641     5    0.940000  0.054772   
                 128.0         2.375000  1.060660     8    1.093750  0.278308   
          0.0003 32.0          1.000000  1.069045    15    1.056667  0.198986   
                 128.0         1.117647  1.317306    17    1.247059  0.232829   
          0.0010 32.0          2.250000  1.500000     4    1.187500  0.239357   
                 128.0         1.555556  0.527046     9    1.227778  0.247628   

                                   
                            count  
optimizer lr     batch_size        
adam      0.0001 32.0           5  
                 128.0          8  
          0.0003 32.0          15  
                 128.0         17  
          0.0010 32.0           4  
                 128.0          9

final_loss                  final_length  \
                                      mean        std count         mean   
optimizer batch_size adam_beta2                                            
adam      32.0       0.900       15.686562   9.575799     8     3.973763   
                     0.990       20.837181  12.438532     8     5.061581   
                     0.999       13.651018   6.186235     8     5.793783   
          128.0      0.900        8.932590   1.280696    11     5.150219   
                     0.990       16.860607   9.472846    12     4.333333   
                     0.999       18.370314  10.713151    11     3.728792   

                                                       lr                  \
                                      std count      mean       std count   
optimizer batch_size adam_beta2                                             
adam      32.0       0.900       2.469403     8  0.000313  0.000295     8   
                     0.990       1.738558     8  0.000513  0.000412     8   
                     0.999       0.537295     8  0.000300  0.000000     8   
          128.0      0.900       0.964312    11  0.000291  0.000255    11   
                     0.990       2.461830    12  0.000408  0.000368    12   
                     0.999       2.469663    11  0.000618  0.000366    11   

                                random_seed                 temperature  \
                                       mean       std count        mean   
optimizer batch_size adam_beta2                                           
adam      32.0       0.900         0.875000  1.356203     8    1.068750   
                     0.990         1.875000  1.125992     8    0.981250   
                     0.999         1.125000  1.125992     8    1.112500   
          128.0      0.900         1.000000  1.414214    11    1.222727   
                     0.990         1.750000  1.138180    12    1.229167   
                     0.999         1.818182  0.873863    11    1.163636   

                                                 
                                      std count  
optimizer batch_size adam_beta2                  
adam      32.0       0.900       0.205178     8  
                     0.990       0.119336     8  
                     0.999       0.241646     8  
          128.0      0.900       0.233841    11  
                     0.990       0.298830    12  
                     0.999       0.218049    11

## Length costs

- hard to conclude!

In [7]:
fields = ['final_loss', 'final_length', 'lr', 'adam_beta2', 'random_seed',
          'optimizer', 'temperature', 'length_cost', 'ada_len_cost_thresh']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['ada_len_cost_thresh', 'length_cost'])
print(a)
a.agg(['mean', 'std', 'count'])

final_loss                  final_length  \
                                      mean        std count         mean   
ada_len_cost_thresh length_cost                                            
0.0                 0.00         18.506088  12.715582     3     6.000000   
                    0.01          9.236023   0.034591     3     6.000000   
                    0.03         34.619301   0.896024     2     1.000000   
                    0.10         25.816776   3.839440     2     1.136364   
                    0.30         26.410374   1.040740     2     1.000000   
5.0                 0.01         10.023862   0.496929     3     6.000000   
                    0.03         13.669459   7.134633     7     5.359721   
                    0.10          7.950337        NaN     1     5.992647   
                    0.30         15.160644   9.867570     9     4.331848   
10.0                0.01         15.566474   9.251262     6     5.143494   
                    0.03          9.345360   1.003552     4     5.970756   
                    0.10          9.728986   1.531022     5     5.872460   
                    0.30         15.060054   8.741286     7     3.422651   
15.0                0.00         39.482071        NaN     1     6.000000   
                    0.03         11.832818        NaN     1     6.000000   
                    0.10         30.623096        NaN     1     1.000000   
                    0.30         10.200574        NaN     1     4.467246   

                                                       lr                  \
                                      std count      mean       std count   
ada_len_cost_thresh length_cost                                             
0.0                 0.00         0.000000     3  0.000767  0.000404     3   
                    0.01         0.000000     3  0.000167  0.000115     3   
                    0.03         0.000000     2  0.001000  0.000000     2   
                    0.10         0.192847     2  0.000300  0.000000     2   
                    0.30         0.000000     2  0.000300  0.000000     2   
5.0                 0.01         0.000000     3  0.000233  0.000115     3   
                    0.03         1.694019     7  0.000600  0.000374     7   
                    0.10              NaN     1  0.000100       NaN     1   
                    0.30         1.982983     9  0.000367  0.000371     9   
10.0                0.01         2.030175     6  0.000533  0.000361     6   
                    0.03         0.052436     4  0.000300  0.000000     4   
                    0.10         0.174641     5  0.000220  0.000110     5   
                    0.30         1.734998     7  0.000214  0.000107     7   
15.0                0.00              NaN     1  0.001000       NaN     1   
                    0.03              NaN     1  0.000300       NaN     1   
                    0.10              NaN     1  0.001000       NaN     1   
                    0.30              NaN     1  0.000300       NaN     1   

                                adam_beta2                 random_seed  \
                                      mean       std count        mean   
ada_len_cost_thresh length_cost                                          
0.0                 0.00          0.960000  0.051962     3    1.333333   
                    0.01          0.960000  0.051962     3    1.333333   
                    0.03          0.994500  0.006364     2    0.500000   
                    0.10          0.999000  0.000000     2    1.000000   
                    0.30          0.990000  0.000000     2    3.000000   
5.0                 0.01          0.930000  0.051962     3    1.666667   
                    0.03          0.997714  0.003402     7    1.714286   
                    0.10          0.900000       NaN     1    0.000000   
                    0.30          0.983000  0.031429     9    2.111111   
10.0                0.01          0.933000  0.051123     6    0.833333   
        

# Best runs

In [13]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss', 
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size', 'vocab_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir', 'receiver_nlayers', 'sender_nlayers',
         'dropout', 'random_seed']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 12.0)
criterion_3 = (df['final_length'] < 5)

# all runs shld be like that, but I prefer to verify, in case I load the wrong runs
# criterion_6 = (df['receiver_mlp'] == 'True')  
# criterion_7 = (df['sender_squash_output'] >= '0.0')
# criterion_8 = (df['sender_mlp'] == 'True') 
# criterion_9 = (df['batch_size'] == '128') 
criterion_10 = (df['entropy_coef'] == 0.0) 


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3 & criterion_10]
a = sub_df[fields].groupby(['ada_len_cost_thresh', 'length_cost',     
                            'lr', 'vocab_size', 'temperature']) 
print(a)
a.agg(['mean', 'std', 'count'])

final_length  \
                                                                      mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature                
5.0                 0.3         0.0001 64.0       0.90            4.573529   
                                0.0003 64.0       1.25            4.268048   
10.0                0.3         0.0001 64.0       0.90            4.397059   
                                0.0003 64.0       1.25            3.868316   
15.0                0.3         0.0003 512.0      1.00            4.467246   

                                                                               \
                                                                    std count   
ada_len_cost_thresh length_cost lr     vocab_size temperature                   
5.0                 0.3         0.0001 64.0       0.90              NaN     1   
                                0.0003 64.0       1.25              NaN     1   
10.0                0.3         0.0001 64.0       0.90         0.000000     2   
                                0.0003 64.0       1.25         0.077517     2   
15.0                0.3         0.0003 512.0      1.00              NaN     1   

                                                              final_loss  \
                                                                    mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature              
5.0                 0.3         0.0001 64.0       0.90         10.561565   
                                0.0003 64.0       1.25          8.299747   
10.0                0.3         0.0001 64.0       0.90         10.627255   
                                0.0003 64.0       1.25          9.737232   
15.0                0.3         0.0003 512.0      1.00         10.200574   

                                                                               \
                                                                    std count   
ada_len_cost_thresh length_cost lr     vocab_size temperature                   
5.0                 0.3         0.0001 64.0       0.90              NaN     1   
                                0.0003 64.0       1.25              NaN     1   
10.0                0.3         0.0001 64.0       0.90         0.000000     2   
                                0.0003 64.0       1.25         0.058396     2   
15.0                0.3         0.0003 512.0      1.00              NaN     1   

                                                              final_length_1.0  \
                                                                          mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature                    
5.0                 0.3         0.0001 64.0       0.90                4.069620   
                                0.0003 64.0       1.25                3.738397   
10.0                0.3         0.0001 64.0       0.90                4.062236   
                                0.0003 64.0       1.25                3.308017   
15.0                0.3         0.0003 512.0      1.00                4.229958   

                                                                               \
                                                                    std count   
ada_len_cost_thresh length_cost lr     vocab_size temperature                   
5.0                 0.3         0.0001 64.0       0.90              NaN     1   
                                0.0003 64.0       1.25              NaN     1   
10.0                0.3         0.0001 64.0       0.90         0.000000     2   
                                0.0003 64.0       1.25         0.086524     2   
15.0                0.3         0.0003 512.0      1.00              NaN     1   

                                                              final_length_2.0  \
                                                                          mean   
ada_len_cost_thresh length_cost lr     vocab_size temperatur

## Find a specific run

In [15]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss',
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 10.0)
criterion_3 = (df['final_length'] < 4)


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3]
print(sub_df)
print(sub_df['checkpoint_dir'].values)

    ada_H_cost_thresh  ada_len_cost_thresh  adam_beta2  batch_size  \
39                0.0                 10.0         0.9       128.0   
42                0.0                 10.0         0.9       128.0   

                                       checkpoint_dir  checkpoint_freq  cuda  \
39  /Tmp/slurm.935601.0/bosctom/proto_2_adam_sd0/3...              0.0  True   
42  /Tmp/slurm.934965.0/bosctom/proto_2_adam_sd0/9...              0.0  True   

    dataset_seed        device                      distributed_context  rank  \
39           0.0  device(type=  DistributedContext(is_distributed=False   0.0   
42           0.0  device(type=  DistributedContext(is_distributed=False   0.0   

    local_rank  world_size mode  distributed_port  dropout  entropy_coef  \
39         0.0         1.0   gs           18363.0      0.2           0.0   
42         0.0         1.0   gs           18363.0      0.2           0.0   

     fp16  length_cost load_from_checkpoint      lr  max_len  momentum  \
3